In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

furPattern_load = pd.read_csv("Tree_Data_Range_5000_CSV.csv",delimiter=",",header=None) # Load the entire CSV file
x_furPattern_train = furPattern_load.values[:2500,2] # Takes the first 2,500 x-values for the fur pattern. In "Column C" excel.
y_furPattern_train = furPattern_load.values[:2500,3] # Takes the first 2,500 y-values for the fur pattern. In "Column D" excel.

x_test = furPattern_load.values[2500:5000,2] # Takes the last 2,500 x-values.
y_test = furPattern_load.values[2500:5000,3] # Takes the last 2,500 y-values.


# One hot encoding
dummy_y_train = np_utils.to_categorical(y_furPattern_train)
dummy_y_train = np.delete(dummy_y_train, obj=0,axis=1)

# Function that creates our neural network for the cat weights
def weights_NN():
    # Creating model
    model = Sequential()
   
    # Adding input layer with rectified linear activation function for input layer
    model.add(Dense(8, input_dim=1, activation='relu'))
    
    # Adding output layer with softmax activation function
    model.add(Dense(4, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

'''
estimator = KerasClassifier(build_fn=weights_NN, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, x_furPattern_train, dummy_y_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
# Printing with this gets "Baseline: 99.50% (0.35%)"
'''

'''
Without using estimator, testing the NN with the same 5000 training points and using the testAccuracy()
function returns estimates of ~99.50%.

Without using estimator, training the NN with 2500 points and testing with the other 2500 points and
using the testAccuracy() function returns estimates of ~99.60%

Only benefit to not using estimator is that the total runtime is reduced SUBSTANTIALLY.
'''

model = weights_NN()
model.fit(x_furPattern_train, dummy_y_train, epochs=200, batch_size=15, verbose=0)

## Function for predicting labels using keras predict function
def predict(model,x):
    preds = np.argmax(model.predict(x), axis=-1)
    #preds = model.predict_classes(x)
    for i in range(len(preds)):
        preds[i] = preds[i] + 1
    return preds

## Function for testing accuracy of predictions
def testAccuracy(preds, labels):
    count = 0
    for i in range(len(preds)):
        if(preds[i] == labels[i]):
            count += 1
    final = 100*(count/len(labels))
    return print("Accuracy: %.2f" % (final))

predictions = predict(model,x_test)
testAccuracy(predictions,y_test)

Accuracy: 99.20
